In [6]:
import numpy as np
import pandas as pd 
import os

In [21]:
#Credentials set up

#Local
from dotenv import load_dotenv
load_dotenv('../.env',override=True)

CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#Kaggle
# from kaggle_secrets import UserSecretsClient
# CLIENT_ID = UserSecretsClient().get_secret("SPOTIFY_CLIENT_ID") 
# CLIENT_SECRET = UserSecretsClient().get_secret("SPOTIFY_CLIENT_SECRET")


In [9]:
pd.set_option('display.max_columns', None)
spotify_youtube = pd.read_csv('../data/input/Spotify_Youtube.csv')
spotify_youtube.columns = spotify_youtube.columns.str.lower()

spotify_youtube['spotify_track_id'] = spotify_youtube['uri'].str.split(":").str[2]
spotify_youtube['spotify_artist_id'] = spotify_youtube['url_spotify'].str.replace('https://open.spotify.com/artist/','')

spotify_youtube.sort_values(by='views', ascending=False).head()

,unnamed: 0,artist,url_spotify,track,album,album_type,uri,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,url_youtube,title,channel,views,likes,comments,description,licensed,official_video,stream,spotify_track_id,spotify_artist_id
1147,1147,Luis Fonsi,https://open.spotify.com/artist/4V8Sr092TqfHkf...,Despacito,VIDA,album,spotify:track:6habFhsOp2NvshLv26DqMb,0.655,0.797,2.0,-4.787,0.1530,0.198,0.000000,0.0670,0.839,177.928,229360.0,https://www.youtube.com/watch?v=kJQP7kiw5Fk,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,8.079649e+09,50788652.0,4252791.0,“Despacito” disponible ya en todas las platafo...,True,True,1.506598e+09,6habFhsOp2NvshLv26DqMb,4V8Sr092TqfHkfAA5fXXqG
365,365,Daddy Yankee,https://open.spotify.com/artist/4VMYDCV2IEDYJA...,Despacito,VIDA,album,spotify:track:6habFhsOp2NvshLv26DqMb,0.655,0.797,2.0,-4.787,0.1530,0.198,0.000000,0.0670,0.839,177.928,229360.0,https://www.youtube.com/watch?v=kJQP7kiw5Fk,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,8.079647e+09,50788626.0,4252791.0,“Despacito” disponible ya en todas las platafo...,True,True,1.506598e+09,6habFhsOp2NvshLv26DqMb,4VMYDCV2IEDYJArk749S6m
12452,12452,Ed Sheeran,https://open.spotify.com/artist/6eUKZXaKkcviH0...,Shape of You,÷ (Deluxe),album,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.825,0.652,1.0,-3.183,0.0802,0.581,0.000000,0.0931,0.931,95.977,233713.0,https://www.youtube.com/watch?v=JGwWNGJdvx8,Ed Sheeran - Shape of You (Official Music Video),Ed Sheeran,5.908398e+09,31047780.0,1130327.0,The official music video for Ed Sheeran - Shap...,True,True,3.362005e+09,7qiZfU4dY1lWllzX7mPBI3,6eUKZXaKkcviH0Ku9w2n3V
14580,14580,Charlie Puth,https://open.spotify.com/artist/6VuMaDnrHyPL1p...,See You Again (feat. Charlie Puth),See You Again (feat. Charlie Puth),single,spotify:track:2JzZzZUQj3Qff7wapcbKjc,0.689,0.481,10.0,-7.503,0.0815,0.369,0.000001,0.0649,0.283,80.025,229526.0,https://www.youtube.com/watch?v=RgKAFK5djSk,Wiz Khalifa - See You Again ft. Charlie Puth [...,Wiz Khalifa Music,5.773798e+09,40147674.0,2127346.0,Download the new Furious 7 Soundtrack Deluxe V...,True,True,1.521255e+09,2JzZzZUQj3Qff7wapcbKjc,6VuMaDnrHyPL1p4EHjYLi7
12469,12469,Wiz Khalifa,https://open.spotify.com/artist/137W8MRPWKqSmr...,See You Again (feat. Charlie Puth),See You Again (feat. Charlie Puth),single,spotify:track:2JzZzZUQj3Qff7wapcbKjc,0.689,0.481,10.0,-7.503,0.0815,0.369,0.000001,0.0649,0.283,80.025,229526.0,https://www.youtube.com/watch?v=RgKAFK5djSk,Wiz Khalifa - See You Again ft. Charlie Puth [...,Wiz Khalifa Music,5.773797e+09,40147618.0,2127345.0,Download the new Furious 7 Soundtrack Deluxe V...,True,True,1.521255e+09,2JzZzZUQj3Qff7wapcbKjc,137W8MRPWKqSmrBGDBFSop


In [23]:
# ENRICHMENT - ONLY RUN ONCE
import time
import math
import requests
import base64
from typing import Iterable, List, Dict

SPOTIFY_TOKEN_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_API_BASE = "https://api.spotify.com/v1"

def get_access_token(client_id: str, client_secret: str) -> str:
    r = requests.post(
        SPOTIFY_TOKEN_URL,
        data={"grant_type": "client_credentials"},
        auth=(client_id, client_secret),
        timeout=15
    )
    r.raise_for_status()
    return r.json()["access_token"]


def chunk(iterable: Iterable, size: int):
    it = list(iterable)
    for i in range(0, len(it), size):
        yield it[i:i+size]

def spotify_get(url: str, headers: Dict, params: Dict = None, max_retries: int = 5):
    for attempt in range(max_retries):
        resp = requests.get(url, headers=headers, params=params, timeout=30)
        if resp.status_code == 429:
            wait = int(resp.headers.get("Retry-After", "5"))
            time.sleep(wait + 1)
            continue
        resp.raise_for_status()
        return resp.json()
    raise RuntimeError(f"Too many 429s on {url}")


def fetch_tracks(track_ids: List[str], headers: Dict):
    out = []
    i = 1
    for batch in chunk(track_ids, 50):
        print('running batch ',i)
        data = spotify_get(f"{SPOTIFY_API_BASE}/tracks", headers, {"ids": ",".join(batch)})
        out.extend(data["tracks"])
        i = i + 1
    return out


def fetch_artists(artist_ids: List[str], headers: Dict):
    out = []
    for batch in chunk(artist_ids, 50):
        data = spotify_get(f"{SPOTIFY_API_BASE}/artists", headers, {"ids": ",".join(batch)})
        out.extend(data["artists"])
    return out


token = get_access_token(CLIENT_ID, CLIENT_SECRET)
headers = {"Authorization": f"Bearer {token}"}

In [27]:
if os.path.exists('../data/artists.csv'):
    artists_df = pd.read_csv('../data/artists.csv')
else:
    artist_ids = spotify_youtube['spotify_artist_id'].unique()
    fetched_arist_data = fetch_artists(artist_ids, headers)

    rows = []

    for artist in fetched_arist_data:
        rows.append({
            'spotify_artist_name': artist.get('name'),
            'spotify_artist_id': artist.get('id'),
            'spotify_artist_followers': (artist.get('followers') or {}).get('total'),
            'spotify_artist_genres': ",".join(artist.get('genres'))
        })

    artists_df = pd.DataFrame(rows)
    artists_df.to_csv('../data/artists.csv', index=False)

In [28]:
if os.path.exists('../data/tracks.csv'):
    tracks_df = pd.read_csv('../data/tracks.csv')
else:
    track_ids = spotify_youtube['spotify_track_id'].unique()
    fetched_track_data = fetch_tracks(track_ids, headers)

    rows = []

    for track in fetched_track_data:
        rows.append({
            'spotify_track_name': track.get('name'),
            'spotify_track_id': track.get('id'),
            'spotify_album_name': (track.get('album') or {}).get('name'),
            'spotify_album_release_date': (track.get('album') or {}).get('release_date'),
            'spotify_track_is_explicit': track.get('explicit'),
            'spotify_track_isrc': (track.get('external_ids') or {}).get('isrc'),
            'spotify_track_popularity': track.get('popularity'),
        })


    tracks_df = pd.DataFrame(rows)
    tracks_df.to_csv('../data/tracks.csv', index=False)

In [29]:
spotify_youtube = (spotify_youtube.sort_values(["spotify_track_id", "views"], ascending=[True, False])
             .drop_duplicates(subset="spotify_track_id", keep="first")
             .reset_index(drop=True))

spotify_youtube_tracks_enriched = spotify_youtube.merge(tracks_df, on="spotify_track_id",suffixes=("","_track"))

spotify_youtube_tracks_artists_enriched = spotify_youtube_tracks_enriched.merge(artists_df, on="spotify_artist_id",suffixes=("","_artist"))

In [30]:
main_df = spotify_youtube_tracks_artists_enriched

s = main_df["spotify_album_release_date"].astype(str)
year_only = s.str.fullmatch(r"\d{4}")
s.loc[year_only] = s.loc[year_only] + "-01-01"
s = s.replace("0000", np.nan)
main_df['album_release_date'] = pd.to_datetime(s, errors="coerce")
main_df['album_release_year'] = main_df['album_release_date'].dt.year

main_df['track_is_explicit'] = main_df['spotify_track_is_explicit'].astype(int)

In [31]:
def fixed_deciles_fast(s, n=10):
    pct = s.rank(pct=True, method='max')
    dec = np.ceil(pct * n).astype('Int64') 
    return dec.clip(upper=n)

deciles_features_list = ['danceability', 'energy', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence','views']
direct_use_features = ['loudness','tempo','duration_ms']

for feature in deciles_features_list + direct_use_features:
    main_df[feature+'_decile'] = fixed_deciles_fast(main_df[feature])
    print(feature + ' deciles created')

# debug duplicate bins/skewed features
# import matplotlib.pyplot as plt
# for feature in deciles_features_list:
#     print(feature)
#     # main_df[feature].hist()
#     main_df[feature + '_decile'].hist()
#     plt.suptitle('Histograms of ' + feature)
#     plt.tight_layout()
#     plt.show()

danceability deciles created
energy deciles created
speechiness deciles created
acousticness deciles created
instrumentalness deciles created
liveness deciles created
valence deciles created
views deciles created
loudness deciles created
tempo deciles created
duration_ms deciles created


In [ ]:
main_df.to_csv('../data/main_df.csv', index=False)